<h1>ASSIGNEMENT 2</h1>

<h3>Zahra Aslam Khan | Section A</h3>

<p>Code Challenge 1: Implement the Median String Algorithm which Identifies the
median string of length k in a collection of longer strings.</p>

In [ ]:
import time
import itertools
from collections import Counter, defaultdict

def read_file(filename):
    with open(filename, "r") as file:
        data = [line.strip() for line in file.readlines()]
    return int(data[0]), data[1:]

def generate_kmers(sequence, k):
    return {sequence[i:i + k] for i in range(len(sequence) - k + 1)}

def hamming_distance(seq1, seq2):
    return sum(1 for a, b in zip(seq1, seq2) if a != b)

def all_possible_kmers(k):
    return ["".join(p) for p in itertools.product("ACTG", repeat=k)]

def compute_distance(kmer, dna_string, k):
    return min(hamming_distance(kmer, sub_kmer) for sub_kmer in generate_kmers(dna_string, k))

def kmer_distance_across_dna(kmer, dna_sequences, k):
    return [(kmer, (seq, compute_distance(kmer, seq, k))) for seq in dna_sequences]

def compute_all_kmer_distances(dna_sequences, k):
    all_kmers = all_possible_kmers(k)
    distance_list = list(itertools.chain(*[kmer_distance_across_dna(kmer, dna_sequences, k) for kmer in all_kmers]))
    distance_dict = defaultdict(list)
    for kmer, value in distance_list:
        distance_dict[kmer].append(value)
    return {k: sorted(v, key=lambda x: x[1]) for k, v in distance_dict.items()}

def find_median_string(dna_sequences, k):
    kmer_distances = compute_all_kmer_distances(dna_sequences, k)
    return min(kmer_distances.items(), key=lambda x: sum(dist[1] for dist in x[1]))[0]

def process_file(filename):
    k, dna_sequences = read_file(filename)
    return find_median_string(dna_sequences, k)

if __name__ == "__main__":
    input_filename = "input_file.txt"  
    result = process_file(input_filename)
    print(result)


<p>Code Challenge 2: Implement the Greedy Motif Search algorithm.</p>

In [ ]:
import sys
import timeit
import regex
import heapq
import operator
import numpy as np
from itertools import combinations, product, chain
from collections import defaultdict, Counter
from functools import reduce

def load_input(file_path):
    with open(file_path) as file:
        lines = [line.strip() for line in file.readlines()]
    k, t = map(int, lines[0].split())
    return k, t, lines[1:]

def get_kmers(sequence, k):
    return [sequence[i:i + k] for i in range(len(sequence) - k + 1)]

def generate_kmers(k, dna_sequences):
    return [get_kmers(seq, k) for seq in dna_sequences]

def compute_profile_and_consensus(motifs, k, t):
    motif_array = np.array([list(motif) for motif in motifs])
    consensus = ''
    score = 0
    profile_matrix = np.zeros((4, k))
    base_order = ['A', 'C', 'G', 'T']

    for i in range(k):
        base_counts = Counter(motif_array[:, i])
        most_common_base, most_common_count = base_counts.most_common(1)[0]
        consensus += most_common_base
        score += t - most_common_count
        
        # Ensure all bases are considered (defaulting to zero if missing)
        for base in base_order:
            count = base_counts.get(base, 0)
            profile_matrix[base_order.index(base), i] = count / float(t)

    return consensus, score, profile_matrix
def calculate_kmer_probability(kmer, base_order, profile_matrix):
    probabilities = [profile_matrix.item(base_order.index(kmer[i]), i) for i in range(len(kmer))]
    return reduce(operator.mul, probabilities, 1)

def update_motifs(current_motifs, kmer_candidates, k, t):
    base_order = ['A', 'C', 'G', 'T']
    consensus, score, profile_matrix = compute_profile_and_consensus(current_motifs, k, t)
    probabilities = [(kmer, calculate_kmer_probability(kmer, base_order, profile_matrix)) for kmer in kmer_candidates]
    current_motifs.append(heapq.nlargest(1, probabilities, key=lambda x: x[1])[0][0])
    return current_motifs

def refine_motifs(initial_motifs, kmer_lists, k, t):
    for kmer_list in kmer_lists:
        initial_motifs = update_motifs(initial_motifs, kmer_list, k, t)
    return initial_motifs

def find_best_motifs(k, t, dna_sequences):
    kmer_combinations = generate_kmers(k, dna_sequences)
    return [refine_motifs([initial], kmer_combinations[1:], k, t) for initial in kmer_combinations[0]]

def greedy_motif_search(dna_sequences, k, t):
    kmer_combinations = generate_kmers(k, dna_sequences)
    best_motifs = [seq[:k] for seq in dna_sequences]
    best_score = compute_profile_and_consensus(best_motifs, k, t)[1]
    candidate_motifs = find_best_motifs(k, t, dna_sequences)
    
    for motifs in candidate_motifs:
        consensus, score, _ = compute_profile_and_consensus(motifs, k, t)
        if score < best_score:
            best_score = score
            best_motifs = motifs
    
    return best_motifs

def process_file(file_name):
    k, t, dna_sequences = load_input(file_name)
    return greedy_motif_search(dna_sequences, k, t)

if __name__ == "__main__":
    input_file = "input_file2.txt" 
    output_motifs = process_file(input_file)
    
    for motif in output_motifs:
        print(motif)
    
